In [1]:
# Importação de bibliotecas

import os
import sys
import pandas as pd
import numpy as np
import polars as pl
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from bertopic import BERTopic
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset
from umap import UMAP

c:\Users\csouza\AppData\Local\pypoetry\Cache\virtualenvs\academic-fingerprint-4AictD0y-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Definicao da raiz do projeto

PROJECT_ROOT = 'G:/Csouza/nlp/topic_modeling'

os.chdir(PROJECT_ROOT)

sys.path.insert(0, PROJECT_ROOT)

In [3]:
def extract(extract_path, file_name='all_process.xlsx', sheet_name='Sheet1'):
    
    return pl.read_excel(f'{extract_path}/{file_name}', sheet_name=sheet_name)

In [4]:
data_path = os.path.join(PROJECT_ROOT, 'data', 'internal', 'fapesp_projects')

full_data = extract(data_path)

variables = {
'N. Processo_B.V': 'n_processo',
'Data de Início': 'data',
'Título (Português)': 'titulo',
'Grande Área do Conhecimento': 'grande_area',
'Área do Conhecimento': 'area',
'Subárea do Conhecimento': 'subarea',
'Palavras-Chave do Processo': 'palavras_chave',
'Assuntos': 'assuntos',
'Resumo (Português)': 'resumo'}

full_data = (
    full_data
    .lazy()
    .rename(variables)
    .select(variables.values())
    .filter(
        pl.col('n_processo').is_not_null(),
        pl.col('resumo').is_not_null(),
        pl.col('resumo') != '')
    .with_columns(
        pl.col('data').str.to_datetime('%m-%d-%y').dt.year().alias('ano'),
        pl.col('data').str.to_datetime('%m-%d-%y').dt.month().alias('mes'))
    .select(pl.exclude('data'))
).collect()

full_data.head(3)

n_processo,titulo,grande_area,area,subarea,palavras_chave,assuntos,resumo,ano,mes
str,str,str,str,str,str,str,str,i32,i8
"""95/04916-0""","""Estudo sistemático de campos h…","""Ciências Exatas e da Terra""","""Física""","""Física da Matéria Condensada""","""CORRELACAO ANGULAR, ESTUDO SIS…",null,"""Este projeto está vinculado ao…",1995,12
"""95/05064-7""","""Cultura, ideologia e represent…","""Ciências Humanas""","""Sociologia""","""Outras Sociologias Específicas""","""BRASIL, IDENTIDADE, PENSAMENTO…","""Brasil:Identidade social""","""Participar do Seminário """"Soci…",1995,12
"""95/09836-4""","""Bernard Schmitt | Université d…","""Ciências Exatas e da Terra""","""Probabilidade e Estatística""","""Probabilidade""","""COMPRESSOR, ENTROPIA, ESTADO D…","""Entropia (matemática aplicada)…","""O principal objetivo da visita…",1995,12


In [5]:
data_train_test = full_data.filter(pl.col('assuntos').is_not_null(), pl.col('area') == 'Medicina')

data_train_test.shape

(17342, 10)

In [6]:
def get_spacy_model(model='en_core_web_sm'):
    """
    Baixa o modelo de linguagem spaCy se não estiver presente.
    """
    try:
        nlp = spacy.load(model)
    except OSError:
        from spacy.cli import download
        download(model)
        nlp = spacy.load(model)
    return nlp

# Carregar o modelo de linguagem em português do spaCy
nlp = get_spacy_model('pt_core_news_sm')

# Definir as stop words em português usando spaCy
stop_words = nlp.Defaults.stop_words

# Compilador para remover caracteres especiais (exceto acentos e espaços)
special_char_remover = re.compile(r'[^A-Za-zÀ-ÿ\s]')

def clean_text(text):
    if not isinstance(text, str):
        raise ValueError("O argumento 'text' deve ser uma string.")
    
    # Remover caracteres especiais
    text = special_char_remover.sub('', text)
    
    # Tokenizar o texto e remover stop words
    tokens = [token.text for token in nlp(text) if token.text not in stop_words]
    
    # Lematizar o texto
    doc = nlp(' '.join(tokens))
    text = ' '.join([token.lemma_ for token in doc])

    return text

# Carregar os dados
data = data_train_test.to_pandas()

data['titulo'] = data['titulo'].astype(str)
data['palavras_chave'] = data['palavras_chave'].astype(str)

data['cleaned_text'] = data['resumo'].apply(clean_text)
data['cleaned_text'] += ' Título: ' + data['titulo'].apply(clean_text) + ' Palavras-chave: ' + data['palavras_chave'].apply(clean_text)

In [ ]:
# Carregar o tokenizer e o modelo pré-treinado
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
bert_model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Tokenizar os dados
def tokenize_function(examples):
    return tokenizer(examples['cleaned_text'], padding="max_length", truncation=True)

dataset = Dataset.from_pandas(data)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Dividir em conjunto de treino e teste
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)

train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
def save_model(model, save_directory, model_name='bert_model.pt'):
    os.makedirs(save_directory, exist_ok=True)
    model_path = os.path.join(save_directory, model_name)
    
    torch.save(model.state_dict(), model_path)

def save_tokenizer(tokenizer, save_directory, tokenizer_name='tokenizer'):
    os.makedirs(save_directory, exist_ok=True)
    tokenizer_path = os.path.join(save_directory, tokenizer_name)
    
    tokenizer.save_pretrained(tokenizer_path)

def train_model(model, tokenizer, train_dataset, test_dataset, output_dir=None, num_train_epochs=3, per_device_train_batch_size=8, per_device_eval_batch_size=8, warmup_steps=500, weight_decay=0.01, logging_dir=None, logging_steps=10, model_save_dir='./models', model_name='bert_model.pt', tokenizer_save_dir='./models', tokenizer_name='tokenizer', save_logs=False, save_results=False):
    """
    Treina o modelo BERT fine-tuned e salva o modelo e o tokenizer.
    
    Args:
        model: O modelo BERT a ser treinado.
        tokenizer: O tokenizer associado ao modelo.
        train_dataset: O dataset de treinamento.
        test_dataset: O dataset de validação.
        output_dir: Diretório para salvar os resultados do treinamento. Se None, define um valor padrão.
        num_train_epochs: Número de épocas de treinamento.
        per_device_train_batch_size: Tamanho do batch de treinamento por dispositivo.
        per_device_eval_batch_size: Tamanho do batch de avaliação por dispositivo.
        warmup_steps: Número de passos de aquecimento.
        weight_decay: Decaimento de peso (weight decay) para o otimizador.
        logging_dir: Diretório para salvar os logs. Se None, define um valor padrão.
        logging_steps: Passos de logging.
        model_save_dir: Diretório para salvar o modelo fine-tuned.
        model_name: Nome do arquivo do modelo.
        tokenizer_save_dir: Diretório para salvar o tokenizer.
        tokenizer_name: Nome do subdiretório onde o tokenizer será salvo.
        save_logs: Booleano para decidir se os logs devem ser salvos.
        save_results: Booleano para decidir se os resultados devem ser salvos.
    
    Returns:
        trainer: O objeto Trainer após o treinamento.
    """
    # Definir diretórios padrão se None for fornecido
    if output_dir is None:
        output_dir = './results'
    if logging_dir is None:
        logging_dir = './logs'
    
    # Criar os diretórios de saída se não existirem e se for necessário salvar
    if save_results:
        os.makedirs(output_dir, exist_ok=True)
    if save_logs:
        os.makedirs(logging_dir, exist_ok=True)

    # Definir argumentos de treinamento
    training_args = TrainingArguments(
        output_dir=output_dir if save_results else None,
        evaluation_strategy="epoch",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        logging_dir=logging_dir if save_logs else None,
        logging_steps=logging_steps,
    )

    # Definir o Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    # Treinar o modelo
    trainer.train()

    # Salvar o modelo e o tokenizer
    save_model(model, model_save_dir, model_name)
    save_tokenizer(tokenizer, tokenizer_save_dir, tokenizer_name)
    
    return trainer

def evaluate_model(trainer, test_dataset, metric_average='weighted'):
    """
    Avalia a qualidade do treinamento do modelo usando o conjunto de dados de teste.
    
    Args:
        trainer: O objeto Trainer que foi usado para treinar o modelo.
        test_dataset: O conjunto de dados de teste.
        metric_average: Tipo de média a ser usada para calcular as métricas (ex: 'weighted', 'macro', 'micro').
    
    Returns:
        metrics: Um dicionário contendo as métricas calculadas.
    """
    # Prever os rótulos do conjunto de teste
    predictions, labels, _ = trainer.predict(test_dataset)
    preds = predictions.argmax(axis=1)
    
    # Calcular métricas
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average=metric_average)
    recall = recall_score(labels, preds, average=metric_average)
    f1 = f1_score(labels, preds, average=metric_average)
    
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }
    
    return metrics

def load_model(save_directory, model_name='bert_model.pt', pretrained_model_name='neuralmind/bert-base-portuguese-cased', model_class=BertForSequenceClassification):
    model_path = os.path.join(save_directory, model_name)
    
    # Carregar o modelo pré-treinado
    model = model_class.from_pretrained(pretrained_model_name)
    model.load_state_dict(torch.load(model_path))
    
    print(f"Modelo carregado de {model_path}")
    
    return model

def load_tokenizer(save_directory, tokenizer_name='tokenizer'):
    tokenizer_directory = os.path.join(save_directory, tokenizer_name)
    
    tokenizer = BertTokenizer.from_pretrained(tokenizer_directory)

    return tokenizer

def extract_embeddings(texts, model, tokenizer, batch_size=8):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    all_embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1)
        all_embeddings.append(batch_embeddings.cpu())
    
    # Concatenar todos os embeddings em um único tensor
    all_embeddings = torch.cat(all_embeddings, dim=0)
    return all_embeddings

def save_embeddings(embeddings, embeddings_path):
    np.save(embeddings_path, embeddings)
    
def load_embeddings(embeddings_path):
    if os.path.exists(embeddings_path):
        return np.load(embeddings_path)
    else:
        raise FileNotFoundError(f'Embeddings file not found at {embeddings_path}')

def train_bertopic(docs, clean_text=None, use_embeddings=False, precomputed_embeddings=None, bert_model=None, tokenizer=None, umap_n_neighbors=15, umap_n_components=5, umap_min_dist=0.0, umap_metric='cosine', vectorizer_ngram_range=(1, 2), stop_words=stop_words):
    # Pré-processar os documentos
    if clean_text:
        docs = [clean_text(doc) for doc in docs]
    
    if not isinstance(stop_words, list):
        stop_words = list(stop_words)
    
    # Configurações padrão do BERTopic
    umap_model = UMAP(n_neighbors=umap_n_neighbors, n_components=umap_n_components, min_dist=umap_min_dist, metric=umap_metric)
    vectorizer_model = CountVectorizer(ngram_range=vectorizer_ngram_range, stop_words=stop_words)
    
    # Check if we should use embeddings
    if use_embeddings:
        embeddings = precomputed_embeddings
        if embeddings is None:
            assert bert_model is not None, "O modelo deve ser fornecido quando use_embeddings=True"
            assert tokenizer is not None, "O tokenizador deve ser fornecido quando use_embeddings=True"
            
            # Extrair embeddings para os documentos em lotes
            embeddings = extract_embeddings(docs, bert_model, tokenizer)
            
            # Criar e treinar o modelo BERTopic usando os embeddings
            topic_model = BERTopic(umap_model=umap_model, vectorizer_model=vectorizer_model)
            topic_model.fit(docs, embeddings)
    else:
        # Criar e treinar o modelo BERTopic usando a configuração padrão
        topic_model = BERTopic(umap_model=umap_model, vectorizer_model=vectorizer_model)
        topic_model.fit(docs)
    
    return topic_model

In [ ]:
model_path = os.path.join(PROJECT_ROOT, 'models')
tokenizer_path = os.path.join(PROJECT_ROOT, 'tokenizers')
model_name = 'bertimbal_model.pt'
tokenizer_name= 'bertimbal_tokenizer'

if not os.path.exists(f'{model_path}/{model_name}'):
    trainer = train_model(
        model=bert_model,
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_steps=10,
        model_save_dir=model_path,
        model_name=model_name,
        tokenizer_save_dir=tokenizer_name,
        tokenizer_name=tokenizer_name
    )

    metrics = evaluate_model(trainer, test_dataset)
    print("Metrics:", metrics)

In [ ]:
bert_model = load_model(model_path, model_name)
tokenizer = load_tokenizer(tokenizer_path, tokenizer_name)

In [ ]:
bertopic_name = f'{model_path}/bertopic_model.pt'

if not os.path.exists(bertopic_name):
    text_train = train_dataset['cleaned_text'].tolist()
    bertopic_model = train_bertopic(docs=text_train)
    bertopic_model.save(bertopic_name)

bertopic_model = BERTopic.load(bertopic_name)
bertopic_model.visualize_topics()

In [ ]:
embeddings_name = f'{model_path}/roberta_embeddings.npy'
if not os.path.exists(embeddings_name):
    text_embeddings = train_dataset['cleaned_text'].tolist()
    embeddings = extract_embeddings(text_embeddings, bert_model, tokenizer)
    save_embeddings(embeddings, embeddings_name)

embeddings = load_embeddings(embeddings_name)

bertopic_roberta_embeddings_name = f'{model_path}/bertopic_roberta_embeddings_model.pt'
if not os.path.exists(bertopic_roberta_embeddings_name):
    text_train = train_dataset['cleaned_text'].tolist()
    bertopic_roberta_embeddings_model = train_bertopic(docs=text_train, use_embeddings=True, precomputed_embeddings=embeddings)
    bertopic_roberta_embeddings_model.save(bertopic_roberta_embeddings_name)

bertopic_roberta_embeddings_model = BERTopic.load(bertopic_roberta_embeddings_name)
bertopic_roberta_embeddings_model.visualize_topics()